# $\rho'$ analysis notes

## Data Acqusition

### Run numbers

### Lego Train

### Data scheme

In [73]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import pandas as ps
import awkward

class UPCFiles:
    ccup8_2015 =  r"/mnt/d/GoogleDrive/Job/cern/Alice/analysis/data/RhoPrime/2015/pvn/ccup8.sel.new.root"
    ccup9_2015 =  r"/mnt/d/GoogleDrive/Job/cern/Alice/analysis/data/RhoPrime/2015/pvn/ccup9.sel.new.root"
    ccup29_2018 = r"/mnt/d/GoogleDrive/Job/cern/Alice/analysis/data/RhoPrime/2018/pvn/ccup29.sel.new.root"
    ccup30_2018 = r"/mnt/d/GoogleDrive/Job/cern/Alice/analysis/data/RhoPrime/2018/pvn/ccup30.sel.new.root"
    ccup31_2018 = r"/mnt/d/GoogleDrive/Job/cern/Alice/analysis/data/RhoPrime/2018/pvn/ccup31.sel.new.root"
    
    ccup8_2015_win  =  r"D:\GoogleDrive\Job\cern\Alice\analysis\data\RhoPrime\2015\pvn\ccup8.sel.new.root"
    ccup9_2015_win  =  r"D:\GoogleDrive\Job\cern\Alice\analysis\data\RhoPrime\2015\pvn\ccup9.sel.new.root"
    ccup29_2018_win =  r"D:\GoogleDrive\Job\cern\Alice\analysis\data\RhoPrime\2018\pvn\ccup29.sel.new.root"
    ccup30_2018_win =  r"D:\GoogleDrive\Job\cern\Alice\analysis\data\RhoPrime\2018\pvn\ccup30.sel.new.root"
    ccup31_2018_win =  r"D:\GoogleDrive\Job\cern\Alice\analysis\data\RhoPrime\2018\pvn\ccup31.sel.new.root"

events = uproot.open(UPCFiles.ccup8_2015_win)['events']
entries = events.numentries
events.show()
events.name, events.title, events.numentries

TrigType                   (no streamer)              asdtype('>i4')
dV0                        (no streamer)              asdtype("('>f4', (6,))")
EnZDC                      (no streamer)              asdtype("('>f4', (6,))")
dAD                        (no streamer)              asdtype("('>f4', (6,))")
vertex                     (no streamer)              asdtype("('>f4', (3,))")
nTracks                    (no streamer)              asdtype('>i4')
nTracklets                 (no streamer)              asdtype('>i4')
eventinfo                  (no streamer)              asdtype("('>i4', (4,))")
TDCa                       (no streamer)              asdtype("('>f4', (4,))")
TDCc                       (no streamer)              asdtype("('>f4', (4,))")
TriggerType                (no streamer)              asdtype("('>i4', (2,))")
dca0                       (no streamer)              asdtype("('>f4', (177,))")
dca1                       (no streamer)              asdtype("('>f4', (177,))")

(b'events', b'events', 6154096)

## Four prongs selection


In [88]:
start = 0
stop  = 300000
data = events.lazyarrays("*", entrystart=start, entrystop=stop, basketcache=uproot.cache.ThreadSafeArrayCache(100 * 1024**2))


In [63]:
itsl0 = (data.HasPointOnITSLayer0 == 1)
itsl1 = (data.HasPointOnITSLayer1 == 1)
selected = itsl0 + itsl1

# del itsl0
# del itsl1

# tpcref = (data["StatusAndTPCRefit"] > 0)
# itcref = (data["StatusAndITSRefit"] > 0)
# selected = selected * tpcref * itcref

# del tpcref
# del itcref

# dca0 = (np.abs(data["dca0"]) < 3)
# dca1 = (np.abs(data["dca1"]) < 3)
# selected = selected * dca0 * dca1

# del dca0
# del dca1

# tpcncls = (data["TPCNcls"] > 50) 
# sigmapion = (np.abs(data["NumberOfSigmasTPCPion"]) < 3)
# selected = selected * tpcncls * sigmapion

# del tpcncls
# del sigmapion



In [108]:
data = events.lazyarrays("*", entrystart=start, entrystop=stop, basketcache=uproot.cache.ThreadSafeArrayCache(10 * 1024**2))
print ('cache size vs chunks len')
print ("10MB", [data.HasPointOnITSLayer0.chunks[i].shape[0] for i in range(len(data.HasPointOnITSLayer0.chunks))])

data = events.lazyarrays("*", entrystart=start, entrystop=stop, basketcache=uproot.cache.ThreadSafeArrayCache(100 * 1024**2))
print ("100MB", [data.HasPointOnITSLayer0.chunks[i].shape[0] for i in range(len(data.HasPointOnITSLayer0.chunks))])

data = events.lazyarrays("*", entrystart=start, entrystop=stop, basketcache=uproot.cache.ThreadSafeArrayCache(1000 * 1024**2))
print ("1000MB", [data.HasPointOnITSLayer0.chunks[i].shape[0] for i in range(len(data.HasPointOnITSLayer0.chunks))])

cache size vs chunks len
10MB [31493, 33047, 32963, 26505, 26865, 27270, 27045, 27450, 28215, 28980, 27900]
100MB [31493, 33047, 32963, 26505, 26865, 27270, 27045, 27450, 28215, 28980, 27900]
1000MB [31493, 33047, 32963, 26505, 26865, 27270, 27045, 27450, 28215, 28980, 27900]


In [9]:
# TODO: why tpcref freq value is 64?

GoodEvents = np.argwhere(np.sum(selected,axis=1)==4)
GoodTracks = np.argwhere(selected[GoodEvents.flatten()]==True).reshape(len(GoodEvents),8)
GoodEventsAndTracks = np.append(GoodEvents,GoodTracks[:,1:len(GoodTracks):2], axis=1) # array([[ 389,    2,    3,    5,    7],...])
GoodEventsNumber = GoodEventsAndTracks[:,0]
GoodTracksNum = GoodEventsAndTracks[:,1:]
print(f'Numer of events with 4pr: {len(GoodEventsNumber)}')


TypeError: sum() got an unexpected keyword argument 'axis'

Transversal momentum distribution for ccup9 trigger:

In [4]:
pxg = px[GoodEventsNumber][select[np.sum(select,axis=1)==4,:]].reshape(len(GoodEventsNumber),4)
pyg = py[GoodEventsNumber][select[np.sum(select,axis=1)==4,:]].reshape(len(GoodEventsNumber),4)
goodCharges = q[GoodEventsNumber][select[np.sum(select,axis=1)==4,:]].reshape(len(GoodEventsNumber),4)
goodChargesEventNumbers = np.argwhere(np.sum(goodCharges,axis=1)==0)
ptMy = np.sqrt(np.sum(np.add(np.sum(pxg[goodChargesEventNumbers], axis=2)**2, np.sum(pyg[goodChargesEventNumbers], axis=2)**2),axis=1))
counts, bins = np.histogram(ptMy,bins=100, range=(0,2))
fig = plt.figure(figsize=(20, 10))
ax = fig.add_axes([0,0,1,1])
fig.suptitle('4pr Pt q = 0', fontsize=16)
plt.style.use(hep.style.ROOT)
ax.errorbar(bins[:-1],counts, 10 * counts \\ counts,fmt='o', color='black',label=f'Entries {np.sum(counts)}')
ax.legend()

SyntaxError: unexpected character after line continuation character (<ipython-input-4-3539e78e251d>, line 11)

## DRAFTS

In [8]:
nTrack = 7
allData = np.concatenate(
    (
        px[:,nTrack].reshape(entries,1),
        py[:,nTrack].reshape(entries,1),
        pz[:,nTrack].reshape(entries,1), 
        itsl0[:,nTrack].reshape(entries,1),
        itsl1[:,nTrack].reshape(entries,1),
        tpcref[:,nTrack].reshape(entries,1),
        itcref[:,nTrack].reshape(entries,1),
        dca0[:,nTrack].reshape(entries,1),
        dca1[:,nTrack].reshape(entries,1),
        tpcncls[:,nTrack].reshape(entries,1),
        sigmtpcpion[:,nTrack].reshape(entries,1)
   ),
    axis=1)
#df = ps.DataFrame(allData, columns=['px', 'py', 'pz', 'itsl0', 'itsl1', 'tpcref', 'itcref', 'dca0', 'dca1', 'tpcncls', 'sigmtpcpion' ])

#df[(df.itsl1 == 1) * (df.itsl0 == 1) * (df.tpcref > 0) * (df.itcref > 0) * ((np.abs(df.dca0) < 3) + (np.abs(df.dca1) < 3)) * (df.tpcncls > 50) * (np.abs(df.sigmtpcpion) < 3)]



In [1]:
import dask as ds
import uproot
# df = ds.dataframe(allData, columns=['px', 'py', 'pz', 'itsl0', 'itsl1', 'tpcref', 'itcref', 'dca0', 'dca1', 'tpcncls', 'sigmtpcpion' ])
ccup9_2015 =  r"D:\GoogleDrive\Job\cern\Alice\analysis\data\RhoPrime\2015\pvn\ccup9.sel.new.root"
ccup8_2015 =  r"D:\GoogleDrive\Job\cern\Alice\analysis\data\RhoPrime\2015\pvn\ccup8.sel.new.root"

df = uproot.daskframe(ccup9_2015, 'events')


UnboundLocalError: local variable 'path' referenced before assignment

In [16]:
import uproot
import awkward1 as ak

data = uproot.open(ccup9_2015)['events']
events = ak.Array([{name: getattr(obj, "_" + name) for name in obj._fields} for obj in data])

TypeError: 'TTree' object is not iterable

In [15]:
data["Px"].array()

AttributeError: 'numpy.ndarray' object has no attribute 'show'